In [1]:
import sys
import os
from pprint import pprint
sys.path.append(os.path.abspath("../../../"))
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display, HTML
from IPython.display import Markdown
import ipywidgets as iw
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
import usure.common.logging as usurelogging
from usure.config import config
from usure.classification.infrastructure import (
     BasicSentenceCleaner 
    ,CsvLabeledCommentsDao
    ,FileWordVectorsRep
    ,FileModelDao)
from usure.classification.core import CnnLab, ClassifierInput, LabReport, WordVectorsService, LabeledComments
import usure.classification.ui.utils as ui

Using TensorFlow backend.


In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = lines => false

<IPython.core.display.Javascript object>

In [4]:
pd.set_option('colheader_justify', 'left')
pd.set_option('display.max_colwidth', 200)

In [5]:
cleaner = BasicSentenceCleaner(config.assets)
comments_dao = CsvLabeledCommentsDao(config.sets, cleaner)
wv_rep = FileWordVectorsRep(config.embeddings)
model_dao = FileModelDao(config.models)
lab = CnnLab(model_dao)
comments = comments_dao.get("train.csv")
test_comments = comments_dao.get("test.csv")
test_inputs = []

In [6]:
def classification_lab_report(word_vector_no, input, lab):
    display(Markdown(f'## {word_vector_no}. Creating models with embeddings: {input.embeddings_name}'))
    lab_report = lab.train_by_stratifiedkfold(input, folds=10)
    return lab_report

def model_architecture_summary(lab_report):
    display(Markdown(f'### Mean Accuracies and losses'))
    display(lab_report.summary)
    display(Markdown(f'### Accuracies and losses by Stratified KFold (folds with balanced class distributions)'))
    model_reports_df = ui.model_reports_to_DataFrame(lab_report.model_reports)
    display(model_reports_df) 
    
def show_statistics_by_model(word_vector_no, word_vector_name, lab_report):
    """Shows statistics by models created by every folds"""
    display(Markdown(f'## {word_vector_no+1}. {word_vector_name}'))
    for fold_no, model_report in enumerate(lab_report.model_reports):
        display(Markdown(f'### Fold number: {fold_no+1}. Name: {model_report.name}'))
        ui.plot_learning_curves_if_epoches(model_report)
        display(Markdown(f'### Training'))
        print(model_report.training.classification_report)
        confu_mtx = model_report.training.confusion_matrix
        display(confu_mtx)
        display(Markdown(f'### Validation'))
        print(model_report.validation.classification_report)
        confu_mtx = model_report.validation.confusion_matrix
        display(confu_mtx)
        display(Markdown(f'### Test'))
        test_metrics, labeled_predictions = lab.test(model_report.name, test_inputs[word_vector_no])
        print(f"Accuracy: {test_metrics.accuracy}")
        print(f"Loss: {test_metrics.loss}")
        print(test_metrics.classification_report)
        display(test_metrics.confusion_matrix)
        test_labeled_comments = test_inputs[word_vector_no].labeled_comments
        labeled_comments = LabeledComments(f"{model_report.name}-{word_vector_name}.cnn.csv",test_labeled_comments.comments ,labeled_predictions)
        comments_dao.save_from_origin(labeled_comments, "test.csv")

In [7]:
#%matplotlib notebook
display(Markdown(f'# Convolutional Neural Network'))
word_vectors = wv_rep.get_all()
word_vector_names = []
lab_reports = []

#word_vectors = [word_vectors.__next__()] #comment!!!!!
for i, word_vector in enumerate(word_vectors):
    if "FBCR2013" in word_vector.name: #erase
        continue
    wv_service = WordVectorsService(word_vector)
    #test_inputs.append(ClassifierInput(test_comments, wv_service))
    input = ClassifierInput(comments, wv_service)
    lab_report = classification_lab_report(i+1, input, lab)
    model_architecture_summary(lab_report)
    #word_vector_names.append(word_vector.name)
    #lab_reports.append(lab_report)
'''   
for i, word_vector_name in enumerate(word_vector_names): 
    lab_report = lab_reports[i]
    show_statistics_by_model(i, word_vector_name, lab_report)
'''
    

# Convolutional Neural Network

## 13. Creating models with embeddings: tweets.txt.usu.16_w.kvs

### Mean Accuracies and losses

,train_acc_mean,train_acc_stdev,val_acc_mean,val_acc_stdev
0,0.903842,0.036859,0.441808,0.034054


### Accuracies and losses by Stratified KFold (folds with balanced class distributions)

...

### Mean Accuracies and losses

,train_acc_mean,train_acc_stdev,val_acc_mean,val_acc_stdev
0,0.92053,0.048849,0.469209,0.054397


### Accuracies and losses by Stratified KFold (folds with balanced class distributions)

,model name,train_acc,train_loss,val_acc,val_loss
0,7cc8c0d5aaaa4aafa948bc54cdc22c83,0.951336,0.265301,0.516949,1.408914
1,5efbb9e138b54d57a1e7f99507b88e74,0.881679,0.357819,0.406780,1.573302
2,34c23a5178e14be08f6d5a78afc9962d,0.930344,0.289683,0.500000,1.367749
3,dcded2ff78294bdc9f519fbfba848072,0.968541,0.217596,0.504274,1.310742
4,f1061efd485443e6b72e38fe79e1f330,0.918970,0.313576,0.435897,1.468291
5,90c670a37767434ba89948b1bdbe5770,0.954242,0.245901,0.478632,1.365536
6,ab2902e5bac34f41bb2fb75de28fc9f5,0.941905,0.283848,0.456897,1.385558
7,2c3530e3c8b34220b08de601625f7e21,0.821905,0.444479,0.413793,1.640612
8,adab8e5981fc48f48604c5bd2622a96c,0.866794,0.398059,0.408696,1.505969
9,f8179950214841d48ce64d5814fe2d39,0.969582,0.205721,0.570175,1.155118


'   \nfor i, word_vector_name in enumerate(word_vector_names): \n    lab_report = lab_reports[i]\n    show_statistics_by_model(i, word_vector_name, lab_report)\n'